In [23]:
#By: Amr Ezzat 
!pip install tpot
# !pip install xgboost

In [24]:
import tpot
#Had a previous error, had to import XGboost model 
# import xgboost 
from tpot import  TPOTClassifier

#importing ML Packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from __future__ import print_function

#Load EDA packages
import pandas as pd
import numpy as np

In [25]:
path = 'https://raw.githubusercontent.com/PaoloMissier/DataScience-class-demos/master/DATA/'
td = pd.read_csv(path + 'titanic.csv', header=0)
pred_feat = 'Class'
seed = 10

In [26]:
td.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [27]:
print ("Rows     : " ,td.shape[0])
print ("Columns  : " ,td.shape[1])

Rows     :  887
Columns  :  8


In [28]:
#Check the data type of variables (Only Numerical values are accepted by TPOT)
td.dtypes

Survived                     int64
Pclass                       int64
Name                        object
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard      int64
Fare                       float64
dtype: object

In [29]:
#Importing the packages we need to convert into numerical values
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder

In [30]:
#We will not change name, we will just get remove it later on 
categorical_columns = ['Sex']
column_trans = make_column_transformer((OrdinalEncoder(), categorical_columns))

titanic_transformed=column_trans.fit_transform(td)

In [31]:
#Copying new data back into td 
titanic_trans = td.copy()
titanic_trans = pd.DataFrame(titanic_transformed, columns=categorical_columns)
td.update(titanic_trans)

In [32]:
#Checking if the conversion has been successful
td

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,1.0,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,0.0,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,0.0,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,0.0,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,1.0,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,1.0,27.0,0,0,13.0000
883,1,1,Miss. Margaret Edith Graham,0.0,19.0,0,0,30.0000
884,0,3,Miss. Catherine Helen Johnston,0.0,7.0,1,2,23.4500
885,1,1,Mr. Karl Howell Behr,1.0,26.0,0,0,30.0000


In [33]:
#Sex has been converted successfully but it is still seen as an object 
td.dtypes

Survived                     int64
Pclass                       int64
Name                        object
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard      int64
Fare                       float64
dtype: object

In [34]:
#Making sure Sex data type is changed 
td.iloc[:, 3] = pd.to_numeric(td.iloc[:, 3], errors='coerce') 

In [35]:
#Sex data type changed successfully. 
td.dtypes

Survived                     int64
Pclass                       int64
Name                        object
Sex                        float64
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard      int64
Fare                       float64
dtype: object

In [36]:
#Checking for missing data
td.isnull().sum()

Survived                   0
Pclass                     0
Name                       0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    0
Fare                       0
dtype: int64

In [37]:
td.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard',
       'Parents/Children Aboard', 'Fare'],
      dtype='object')

In [38]:
# Methods and Attributes 
dir(tpot)

['TPOTClassifier',
 'TPOTRegressor',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_version',
 'base',
 'builtins',
 'config',
 'decorators',
 'driver',
 'export_utils',
 'gp_deap',
 'gp_types',
 'main',
 'metrics',
 'operator_utils',
 'tpot']

In [39]:
#Arrange data (Remove 'Name' and 'Survived', set 'Survived' as the value we want to predict)
td_X = td.drop("Survived", axis=1)
td_X = td_X.drop("Name", axis=1)
td_y = td['Survived']

In [40]:
#Split in train and test
X_train, X_test, y_train, y_test = train_test_split(td_X, td_y, train_size=0.8, test_size=0.2)

In [41]:
#Checking the split was done properly
X_train

,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
271,1,1.0,37.0,0,1,29.7000
516,3,1.0,32.0,0,0,7.8958
832,3,1.0,21.0,0,0,8.6625
400,3,0.0,21.0,1,0,9.8250
236,2,0.0,8.0,0,2,26.2500
...,...,...,...,...,...,...
841,3,1.0,42.0,0,0,7.5500
117,1,1.0,24.0,0,1,247.5208
72,3,1.0,26.0,1,0,14.4542
166,3,0.0,45.0,1,4,27.9000


In [42]:
### AUTOML TPOT INIT/Fit data/Score
tpot =  TPOTClassifier(generations=10, population_size=30, verbosity=2, random_state = 45)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))


Generation 1 - Current best internal CV score: 0.8434821696134254

Generation 2 - Current best internal CV score: 0.8434821696134254

Generation 3 - Current best internal CV score: 0.8434821696134254

Generation 4 - Current best internal CV score: 0.8434821696134254

Generation 5 - Current best internal CV score: 0.8434821696134254

Generation 6 - Current best internal CV score: 0.8434821696134254

Generation 7 - Current best internal CV score: 0.8476775546898413

Generation 8 - Current best internal CV score: 0.8476775546898413

Generation 9 - Current best internal CV score: 0.8519428628508641

Generation 10 - Current best internal CV score: 0.8519428628508641

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.7500000000000001, min_samples_leaf=2, min_samples_split=8, n_estimators=100)
0.797752808988764


In [22]:
print(tpot.score(X_test, y_test))

0.8595505617977528


In [ ]:
tpot.fitted_pipeline_

In [ ]:
tpot.fit(X_train, y_train)

In [ ]:
exctracted_best_model = tpot.fitted_pipeline_.steps[-1][1]

In [ ]:
exctracted_best_model.fit(X_train, y_train) 

In [ ]:
exctracted_best_model.feature_importances_

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
positions= range(exctracted_best_model.feature_importances_.shape[0])
p1 = plt.bar(positions, exctracted_best_model.feature_importances_)
plt.show()

In [ ]:
td.columns

In [ ]:
tpot.export('tpot_titanic_pipeline.py')

In [ ]:
!cat tpot_titanic_pipeline.py